In [18]:
import requests
import re 
import tabula
import pandas as pd
import urllib 
import time

# Read pdf into list of DataFrame
partial_tables = tabula.read_pdf("krausebooklist.pdf", pages='all')
df = pd.concat(partial_tables)

In [45]:
def from_search_results(resp):
    if resp.json().get('numFound'):
        doc = resp.json()['docs'][0]
        return [doc['key'], doc.get('isbn', [None])[0], doc.get('first_sentence'), doc.get('subject')]
    return ['', '', [], []]

def lookup(row):
    try:
        tries_remaining = 5
        while tries_remaining > 0:
            time.sleep(3)
            title = row['Title'].split(":")[0]
            author = row['Author'].split(",")[0]
            solr_qry = urllib.parse.quote(f"{title} {author}")
            url = f"http://openlibrary.org/search.json?q={solr_qry}"
            print(url)
            resp = requests.get(url)
            print(f"{resp.ok} {resp.status_code} {resp.reason}")
            if resp.ok:
                result = from_search_results(resp)
                return result
            if resp.status_code >= 500:
                time.sleep(10)
                tries_remaining -= 1 
            else:
                break
        return ['', '', [], []]
    except Exception as exc:
        print(exc)
        return ['', '', [], []]

df['ol_info'] = df.apply(lookup, axis=1)

http://openlibrary.org/search.json?q=2020%20Black%20Lives%20Matter%20marches%20Markovics
True 200 OK
['/works/OL26280420W', '9781534186354', None, None]
http://openlibrary.org/search.json?q=A%20complicated%20love%20story%20set%20in%20space%20Hutchinson
True 200 OK
['/works/OL21967844W', '1534448551', None, ["Children's fiction"]]
http://openlibrary.org/search.json?q=A%20lesson%20in%20vengeance%20Lee
True 200 OK
['/works/OL24821659W', '9780593305829', None, ['Fiction', 'Sapphic', 'Queer', 'Lesbian', 'Fantasy', 'Young Adult', 'LGBT', 'Thriller', 'Mystery Thriller', "Children's fiction"]]
http://openlibrary.org/search.json?q=As%20far%20as%20you%27ll%20take%20me%20Stamper
True 200 OK
['/works/OL21709787W', '9781547600175', None, ['Accessible book', 'Protected DAISY']]
http://openlibrary.org/search.json?q=Be%20dazzled%20La%20Sala
True 200 OK
['/works/OL21910713W', '1728239141', None, ["Children's fiction"]]
http://openlibrary.org/search.json?q=Black%20Lives%20Matter%20%20Tyner
True 200 OK
[

In [57]:
field_names = ('ol_link', 'isbn', 'first_sentence', 'subjects')
for (idx, field_name) in enumerate(field_names):
    df[field_name] = df.ol_info.apply(lambda lst: lst[idx])

In [158]:
import numpy 
df['id'] = numpy.arange(len(df))

In [160]:
subjects = []
def extract_subjects(row):
    for subj in (row.subjects or []):
        subjects.append((row.id, subj))
df.apply(extract_subjects, axis=1)        

0     None
1     None
2     None
3     None
4     None
      ... 
19    None
20    None
21    None
22    None
23    None
Length: 849, dtype: object

In [164]:
subjects_df = pd.DataFrame(subjects, columns=['book_id', 'subject'])

In [165]:
df['first_sentence_clean'] = df.first_sentence.apply(lambda s: s[0] if s else '')

In [167]:
fields = ['id', 'Title', 'Author', 'Published', 'ol_link', 'isbn', 'first_sentence_clean', ]

In [168]:
final = df[fields]

In [169]:
final = final.rename({'first_sentence_clean': 'first_sentence'}, axis=1)

In [175]:
final = final.set_index('id')

In [178]:
import sqlalchemy as sa
import sqlite3
conn = sqlite3.connect('texas-targeted-books.db')
final.to_sql('book', conn, if_exists='replace', index=True)

In [174]:
subjects_df.to_sql('subject', conn, if_exists='replace', index=False)

In [221]:
def title(row):
    if row.ol_link:
        return f'<a href="https://openlibrary.org{row.ol_link}">{row.Title}</a>'
    else:
        return row.Title

In [223]:
def row_string(row):
    return(f"""<tr><td>{title(row)}</td><td>{row.Author}</td>
        <td>{row.Published}</td><td>{row.isbn}</td>    
        <td>{', '.join(row.subjects or [])}</td>
        </tr>
        """)

In [224]:
rows = "\n".join(df.sort_values(['Title', 'Published']).apply(row_string, axis=1))

In [228]:
with open("README.html", "w") as outfile:
    outfile.write(f"""<html>
<body>
        <h1>Texas-Targeted Books</h1>
        <a href="texas-targeted-books.db">List in relational database format</a>
        (<a href="https://sqlite.org/index.html">SQLite</a>)
        <p>
                The Texas Tribune reported on <a
                        href="https://www.texastribune.org/2021/10/26/texas-school-books-race-sexuality/">
                        Oct. 26, 2021</a>, that Texas State Rep. Matt Krause,
                in his role as chair of the House Committee on General Investigating,
                has opened an official investigation into the presence of hundreds of books
                in Texas schools, and has required all Texas schools to report on the
                books' presence.</p>

        <p>Here is a best-effort presentation of the Rep. Krause list
                in RDBMS and HTML table form, with hyperlinks. ISBN numbers,
                subjects, and links to OpenLibrary
                entries were made by automated query to the OpenLibrary API;
                no guarantees on their accuracy.</p>

        <ul>
                <li><a href="https://static.texastribune.org/media/files/965725d7f01b8a25ca44b6fde2f5519b/krauseletter.pdf">
                                Chair Krause's letter</a></li>
                <li><a href="https://static.texastribune.org/media/files/94fee7ff93eff9609f141433e41f8ae1/krausebooklist.pdf">
                                Chair Krause's list (PDF)</a></li>
                <li><a href="query_open_library.ipynb">Code generating this page</a></li>
        </ul>

    <table>
    <tr><td>Title</td><td>Author</td><td>Year</td><td>ISBN</td><td>Subjects</td></tr>
  {rows}
  </table>
  </body>
  </html>""")